# COMP551 Project3 Group11
我的前面的数据基本上和stephen一样的，也是把train/test x vector化， y通过one-hot encode 一样vector，但我的整个model和stephen是完全不一样的，然后我用的是keras下面的keras.tuner
我主要tune的是 多少层hidden layer， 和每层hidden layer的nodes的数量，
然后我还加了drop out，也tune了
我尝试改了optimizer， 用了adam 还有 stg，发现没什么差别，所以那一块就没有动
我用的是10cross validation



Loading data MLP to CIFAR10

### Training

In [3]:
!pip install tensorflow==1.14
!pip install keras-tuner
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
import numpy as np
!pip install keras
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print("Shape of training data:")
print(X_train.shape)
print(y_train.shape)
print("Shape of test data:")
print(X_test.shape)
print(y_test.shape)
#examine the dataseet 

# Preparing the dataset
# Transform label indices to one-hot encoded vectors


# Transform images from (32,32,3) to 3072-dimensional vectors (32*32*3)
from keras.utils.np_utils import to_categorical
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train=to_categorical(y_train, num_classes=10)
y_test=to_categorical(y_test, num_classes=10)

X_train /= 255
X_test /= 255
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Shape of training data:
(50000, 32, 32, 3)
(50000, 1)
Shape of test data:
(10000, 32, 32, 3)
(10000, 1)
(50000, 32, 32, 3)
(50000, 10)
(10000, 32, 32, 3)
(10000, 10)


In [4]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam
from keras.activations import relu, elu
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time

LOG_DIR=f"{int(time.time())}"
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))   
    for i in range (hp.Int('Number_of_ConvAndMaxPool', 1, 3)):
      model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
      model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
      model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())


    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    # we can change the 256 to another figure to play with the number of units per layer
    for i in range (hp.Int("n_hidden_layers", 1, 10)):
      model.add(Dense(hp.Int("input_unit2", 128, 256, 64), activation=hp.Choice('activation_type', ['relu'])))
      
    model.add(Dropout(hp.Float('dropout', 0, 0.5, step=0.1, default=0.5)))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(10, activation='softmax'))
    #opt = SGD(lr=0.001, momentum=0.9)
    #optAdam = Adam()
    model.compile(optimizer="sgd",
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

 
tuner = RandomSearch(
    build_model,
    objective = "val_acc",
    max_trials=1,
    executions_per_trial=1,
    directory = LOG_DIR
)

tuner.search(x=X_train, y=y_train, epochs=40, batch_size = 100, validation_split=0.1)
#tuner.search_space_summary()
best_model = tuner.get_best_models(num_models=1)[0]
loss, accuracy = best_model.evaluate(X_test, y_test)
print('test_loss:', loss)
print('test_accuracy:', accuracy)

loss, accuracy = best_model.evaluate(X_train, y_train)
print('train_loss:', loss)
print('train_accuracy:', accuracy)

Train on 45000 samples, validate on 5000 samples
Epoch 1/40


20300/45000 [============>.................] - ETA: 9:23 - loss: 2.3204 - acc: 0.130 - ETA: 8:41 - loss: 2.3172 - acc: 0.145 - ETA: 8:30 - loss: 2.3159 - acc: 0.130 - ETA: 8:25 - loss: 2.3111 - acc: 0.125 - ETA: 8:22 - loss: 2.3116 - acc: 0.122 - ETA: 8:18 - loss: 2.3115 - acc: 0.123 - ETA: 8:22 - loss: 2.3114 - acc: 0.122 - ETA: 8:27 - loss: 2.3094 - acc: 0.130 - ETA: 8:30 - loss: 2.3068 - acc: 0.132 - ETA: 8:33 - loss: 2.3069 - acc: 0.126 - ETA: 8:35 - loss: 2.3067 - acc: 0.122 - ETA: 8:37 - loss: 2.3065 - acc: 0.119 - ETA: 8:38 - loss: 2.3047 - acc: 0.123 - ETA: 8:39 - loss: 2.3042 - acc: 0.120 - ETA: 8:40 - loss: 2.3038 - acc: 0.119 - ETA: 8:41 - loss: 2.3039 - acc: 0.118 - ETA: 8:42 - loss: 2.3033 - acc: 0.121 - ETA: 8:43 - loss: 2.3032 - acc: 0.117 - ETA: 8:45 - loss: 2.3031 - acc: 0.116 - ETA: 8:47 - loss: 2.3022 - acc: 0.117 - ETA: 8:50 - loss: 2.3015 - acc: 0.118 - ETA: 8:53 - loss: 2.3007 - acc: 0.121 - ETA: 8:53 - loss: 2.3004 - acc: 0.123 - ETA: 8:52 - loss: 2.3004 - acc: 0

40700/45000 [==========================>...] - ETA: 5:11 - loss: 2.2125 - acc: 0.178 - ETA: 5:09 - loss: 2.2132 - acc: 0.178 - ETA: 5:08 - loss: 2.2133 - acc: 0.177 - ETA: 5:07 - loss: 2.2136 - acc: 0.177 - ETA: 5:06 - loss: 2.2131 - acc: 0.178 - ETA: 5:04 - loss: 2.2127 - acc: 0.178 - ETA: 5:03 - loss: 2.2120 - acc: 0.178 - ETA: 5:02 - loss: 2.2116 - acc: 0.178 - ETA: 5:01 - loss: 2.2112 - acc: 0.178 - ETA: 4:59 - loss: 2.2108 - acc: 0.179 - ETA: 4:58 - loss: 2.2101 - acc: 0.179 - ETA: 4:57 - loss: 2.2094 - acc: 0.180 - ETA: 4:55 - loss: 2.2090 - acc: 0.180 - ETA: 4:54 - loss: 2.2088 - acc: 0.180 - ETA: 4:53 - loss: 2.2088 - acc: 0.180 - ETA: 4:52 - loss: 2.2088 - acc: 0.180 - ETA: 4:50 - loss: 2.2086 - acc: 0.180 - ETA: 4:49 - loss: 2.2079 - acc: 0.180 - ETA: 4:48 - loss: 2.2073 - acc: 0.180 - ETA: 4:47 - loss: 2.2071 - acc: 0.181 - ETA: 4:45 - loss: 2.2067 - acc: 0.181 - ETA: 4:44 - loss: 2.2062 - acc: 0.181 - ETA: 4:43 - loss: 2.2058 - acc: 0.181 - ETA: 4:42 - loss: 2.2058 - acc: 0

45000/45000 [==============================] - ETA: 54s - loss: 2.1359 - acc: 0.21 - ETA: 52s - loss: 2.1354 - acc: 0.21 - ETA: 51s - loss: 2.1353 - acc: 0.21 - ETA: 50s - loss: 2.1353 - acc: 0.21 - ETA: 48s - loss: 2.1356 - acc: 0.21 - ETA: 47s - loss: 2.1350 - acc: 0.21 - ETA: 46s - loss: 2.1346 - acc: 0.21 - ETA: 45s - loss: 2.1338 - acc: 0.21 - ETA: 43s - loss: 2.1332 - acc: 0.21 - ETA: 42s - loss: 2.1323 - acc: 0.21 - ETA: 41s - loss: 2.1323 - acc: 0.21 - ETA: 39s - loss: 2.1318 - acc: 0.21 - ETA: 38s - loss: 2.1315 - acc: 0.21 - ETA: 37s - loss: 2.1310 - acc: 0.21 - ETA: 36s - loss: 2.1306 - acc: 0.21 - ETA: 34s - loss: 2.1299 - acc: 0.21 - ETA: 33s - loss: 2.1294 - acc: 0.21 - ETA: 32s - loss: 2.1290 - acc: 0.22 - ETA: 30s - loss: 2.1289 - acc: 0.22 - ETA: 29s - loss: 2.1284 - acc: 0.22 - ETA: 28s - loss: 2.1280 - acc: 0.22 - ETA: 27s - loss: 2.1274 - acc: 0.22 - ETA: 25s - loss: 2.1273 - acc: 0.22 - ETA: 24s - loss: 2.1267 - acc: 0.22 - ETA: 23s - loss: 2.1258 - acc: 0.22 - ETA

20400/45000 [============>.................] - ETA: 10:04 - loss: 2.1748 - acc: 0.28 - ETA: 10:07 - loss: 2.1558 - acc: 0.25 - ETA: 10:02 - loss: 2.0460 - acc: 0.29 - ETA: 9:55 - loss: 2.0089 - acc: 0.3000 - ETA: 9:48 - loss: 1.9817 - acc: 0.302 - ETA: 9:43 - loss: 1.9997 - acc: 0.290 - ETA: 9:41 - loss: 2.0475 - acc: 0.284 - ETA: 9:37 - loss: 2.0369 - acc: 0.285 - ETA: 9:33 - loss: 2.0256 - acc: 0.284 - ETA: 9:33 - loss: 2.0103 - acc: 0.290 - ETA: 9:32 - loss: 1.9945 - acc: 0.298 - ETA: 9:32 - loss: 1.9812 - acc: 0.296 - ETA: 9:32 - loss: 1.9662 - acc: 0.306 - ETA: 9:33 - loss: 1.9668 - acc: 0.310 - ETA: 9:30 - loss: 1.9638 - acc: 0.308 - ETA: 9:28 - loss: 1.9680 - acc: 0.307 - ETA: 9:26 - loss: 1.9857 - acc: 0.302 - ETA: 9:24 - loss: 1.9874 - acc: 0.303 - ETA: 9:22 - loss: 1.9779 - acc: 0.310 - ETA: 9:20 - loss: 1.9651 - acc: 0.313 - ETA: 9:18 - loss: 1.9620 - acc: 0.318 - ETA: 9:16 - loss: 1.9614 - acc: 0.316 - ETA: 9:15 - loss: 1.9564 - acc: 0.318 - ETA: 9:14 - loss: 1.9523 - acc: 

...

29760/50000 [================>.............] - ETA: 1:41 - loss: 9.0848e-05 - acc: 1.000 - ETA: 1:41 - loss: 9.1475e-05 - acc: 1.000 - ETA: 1:41 - loss: 9.1396e-05 - acc: 1.000 - ETA: 1:41 - loss: 9.1384e-05 - acc: 1.000 - ETA: 1:41 - loss: 9.1434e-05 - acc: 1.000 - ETA: 1:41 - loss: 9.1366e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1440e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1364e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1272e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1305e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1398e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1318e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1229e-05 - acc: 1.000 - ETA: 1:40 - loss: 9.1160e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1155e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1118e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1075e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.0978e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1050e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1096e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1078e-05 - acc: 1.000 - ETA: 1:39 - loss: 9.1044e-05 

35744/50000 [====================>.........] - ETA: 1:18 - loss: 9.3345e-05 - acc: 1.000 - ETA: 1:18 - loss: 9.3284e-05 - acc: 1.000 - ETA: 1:18 - loss: 9.3236e-05 - acc: 1.000 - ETA: 1:18 - loss: 9.3288e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3289e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3244e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3196e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3207e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3277e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3289e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3377e-05 - acc: 1.000 - ETA: 1:17 - loss: 9.3312e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3386e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3478e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3558e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3527e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3611e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3557e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3530e-05 - acc: 1.000 - ETA: 1:16 - loss: 9.3507e-05 - acc: 1.000 - ETA: 1:15 - loss: 9.3458e-05 - acc: 1.000 - ETA: 1:15 - loss: 9.3382e-05 

41952/50000 [========================>.....] - ETA: 55s - loss: 9.2914e-05 - acc: 1.00 - ETA: 55s - loss: 9.2887e-05 - acc: 1.00 - ETA: 54s - loss: 9.2838e-05 - acc: 1.00 - ETA: 54s - loss: 9.2785e-05 - acc: 1.00 - ETA: 54s - loss: 9.2843e-05 - acc: 1.00 - ETA: 54s - loss: 9.2786e-05 - acc: 1.00 - ETA: 54s - loss: 9.2717e-05 - acc: 1.00 - ETA: 54s - loss: 9.2689e-05 - acc: 1.00 - ETA: 54s - loss: 9.2619e-05 - acc: 1.00 - ETA: 54s - loss: 9.2559e-05 - acc: 1.00 - ETA: 53s - loss: 9.2500e-05 - acc: 1.00 - ETA: 53s - loss: 9.2428e-05 - acc: 1.00 - ETA: 53s - loss: 9.2357e-05 - acc: 1.00 - ETA: 53s - loss: 9.2627e-05 - acc: 1.00 - ETA: 53s - loss: 9.2587e-05 - acc: 1.00 - ETA: 53s - loss: 9.2542e-05 - acc: 1.00 - ETA: 53s - loss: 9.2484e-05 - acc: 1.00 - ETA: 53s - loss: 9.2417e-05 - acc: 1.00 - ETA: 52s - loss: 9.2555e-05 - acc: 1.00 - ETA: 52s - loss: 9.2500e-05 - acc: 1.00 - ETA: 52s - loss: 9.2602e-05 - acc: 1.00 - ETA: 52s - loss: 9.2543e-05 - acc: 1.00 - ETA: 52s - loss: 9.2514e-05 -

48512/50000 [============================>.] - ETA: 31s - loss: 9.1789e-05 - acc: 1.00 - ETA: 30s - loss: 9.1839e-05 - acc: 1.00 - ETA: 30s - loss: 9.1833e-05 - acc: 1.00 - ETA: 30s - loss: 9.1858e-05 - acc: 1.00 - ETA: 30s - loss: 9.1860e-05 - acc: 1.00 - ETA: 30s - loss: 9.1853e-05 - acc: 1.00 - ETA: 30s - loss: 9.1804e-05 - acc: 1.00 - ETA: 30s - loss: 9.1763e-05 - acc: 1.00 - ETA: 30s - loss: 9.1769e-05 - acc: 1.00 - ETA: 29s - loss: 9.1735e-05 - acc: 1.00 - ETA: 29s - loss: 9.1682e-05 - acc: 1.00 - ETA: 29s - loss: 9.1675e-05 - acc: 1.00 - ETA: 29s - loss: 9.1638e-05 - acc: 1.00 - ETA: 29s - loss: 9.1730e-05 - acc: 1.00 - ETA: 29s - loss: 9.1713e-05 - acc: 1.00 - ETA: 29s - loss: 9.1676e-05 - acc: 1.00 - ETA: 29s - loss: 9.1612e-05 - acc: 1.00 - ETA: 28s - loss: 9.1603e-05 - acc: 1.00 - ETA: 28s - loss: 9.1553e-05 - acc: 1.00 - ETA: 28s - loss: 9.1569e-05 - acc: 1.00 - ETA: 28s - loss: 9.1514e-05 - acc: 1.00 - ETA: 28s - loss: 9.1568e-05 - acc: 1.00 - ETA: 28s - loss: 9.1528e-05 -

50000/50000 [==============================] - ETA: 5s - loss: 0.1781 - acc: 0.980 - ETA: 5s - loss: 0.1802 - acc: 0.980 - ETA: 5s - loss: 0.1811 - acc: 0.980 - ETA: 5s - loss: 0.1831 - acc: 0.979 - ETA: 5s - loss: 0.1841 - acc: 0.979 - ETA: 5s - loss: 0.1852 - acc: 0.979 - ETA: 4s - loss: 0.1868 - acc: 0.979 - ETA: 4s - loss: 0.1891 - acc: 0.979 - ETA: 4s - loss: 0.1903 - acc: 0.978 - ETA: 4s - loss: 0.1922 - acc: 0.978 - ETA: 4s - loss: 0.1925 - acc: 0.978 - ETA: 4s - loss: 0.1943 - acc: 0.978 - ETA: 4s - loss: 0.1959 - acc: 0.978 - ETA: 4s - loss: 0.1971 - acc: 0.978 - ETA: 3s - loss: 0.1975 - acc: 0.978 - ETA: 3s - loss: 0.1988 - acc: 0.977 - ETA: 3s - loss: 0.2005 - acc: 0.977 - ETA: 3s - loss: 0.2018 - acc: 0.977 - ETA: 3s - loss: 0.2033 - acc: 0.977 - ETA: 3s - loss: 0.2052 - acc: 0.977 - ETA: 3s - loss: 0.2061 - acc: 0.976 - ETA: 3s - loss: 0.2074 - acc: 0.976 - ETA: 2s - loss: 0.2085 - acc: 0.976 - ETA: 2s - loss: 0.2094 - acc: 0.976 - ETA: 2s - loss: 0.2107 - acc: 0.976 - ETA